<a href="https://colab.research.google.com/github/mbohling/spiking-neuron-model/blob/main/Integrate-and-Fire/SpikingNeuronModel_IAF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#The Spiking Neuron Model - Coding Challenge Problems (Part 1)


#Integrate-and-Fire Spiking Neuron Model
This interactive document is meant to be followed as the reader makes their way through chapter: *The Spiking Neuron Model*.  Each model presented in the chapter will have a section consisting of a step-by-step walkthrough of a simple Python implementation.  This is followed by an interface to run simulations with different parameter values to answer the Coding Challenge Problems.

For each model covered in the chapter, there is a section called **Coding Challenge Problems.**  This is where you will find user-interface components such as value sliders for various parameters.  Use these controls to answer the questions from the text.

**Content Creator**: Maxwell E. Bohling

**Content Reviewer**: Lawrence C. Udeigwe

## How It Works
Google Colab Notebooks have both *Content* cells and *Code* cells. As you progress through the notebook, you MUST make sure to run each code cell as you come to them.  Otherwise, you may run into errors when executing a code cell. Each code cell has a Play button next to it which will execute the code. (Some code may be hidden by default. This is generally because the code is more complex and is not necessary to understand in order to complete the model implementations or to answer the chapter Coding Challenge Problems).

**IMPORTANT**: You have been provided a link to view a **copy** of the original notebooks. You will find that you can edit the content of any cell. If you accidently change a cell, such as a line of code and/or run into errors as you try to run subsequent blocks, simply refresh the page, OR go to the *Runtime menu* and select *Restart runtime*. It is also suggested that you go to the *Edit menu* and select *Clear all outputs*. This will always allow you to revert the notebook to the original version (though you will have to run each code block again.)

For each model covered in the chapter, there is a section called **Coding Challenge Problems**. This is where you will find user-interface components such as value sliders for various parameters. Use these controls to answer the questions from the text.


 Execute the code block. **Initialize Setup**

In [ ]:
#@title Initialize Setup
#@markdown **(No need to understand this code, simply make sure you run this first).**
import sys
import functools as ft
import numpy as np
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import ipywidgets as widgets
import scipy as sc

# [BLOCK TAG: INIT]

try:
  blockSet = [ ]
except:
  print('Something went wrong! Try Refreshing the page.')

blockTags = ['INIT','VP1','NP1','SS1','SS2','SS3','CS1','CS2','VR1']

def pushBlockStack(tag):
    if tag in blockSet:
        return 1
    indx = blockTags.index(tag)
    if len(blockSet) != indx:
        print('ERROR: BLOCK TAG:',tag,'executed out of sequence. Missing BLOCK TAG:', blockTags[indx-1])
        return 0
    else:
        blockSet.append(tag)
        return 1

def printError():
  message = 'Something went wrong!\n\n'  
  message = message + 'Check for the following:\n\n'
  message = message + '\t1.  All previous code blocks have been run the order they appear and output a success message.\n'
  message = message + '\t2.  No other code has been altered.\n\n'
  message = message + 'and then try running the code block again.'
  message = message + ' If there is still an error when executing the code block, try the following:\n\n'
  message = message + '\t1.  Go to the \'Runtime\' menu and select \'Restart Runtime\', then in the \'Edit\' menu, select \'Clear all outputs\'.\n'
  message = message + '\t2.  Refresh the page.\n\n'
  message = message + 'and be sure to run each of the previous code blocks again beginning with \'Initialize Setup\'.\n'
  print(message)
  return 0

def printSuccess(block):
  success = 0
  if len(block) == 0 or pushBlockStack(block) != 0:
    message = 'Success!  Move on to the next section.'
    print(message)
    success = 1
  return success

def checkVoltageParameters(Vrest, Vth, Vreset, Vspike):
  print('Checking Voltage Parameters... ')
  try:
      check_Vrest = Vrest
      check_Vth = Vth
      check_Vreset = Vreset
      check_Vspike = Vspike
  except:
    return 0
  else:
    vals = [Vrest, Vth, Vreset, Vspike]
    correct_vals = [-70, -54, -80, 0]
    if ft.reduce(lambda i, j : i and j, map(lambda m, k: m == k, vals, correct_vals),  False): 
      return 0
  return 1
 
def checkNeuronProperties(Rm, Cm, tau_m):
  print('Checking Neuron Properties... ')
  try:
      check_Rm = Rm
      check_Cm = Cm
      check_tau_m = tau_m
      check_EL = EL
  except:
    return 0
  else:
    vals = [Rm, Cm, tau_m, EL]
    correct_vals = [10, 1, 10, -70]
    if ft.reduce(lambda i, j : i and j, map(lambda m, k: m == k, vals, correct_vals),  False): 
      return 0
  return 1

def checkSimulationSetup(Vrest, Vinitial, EL, t0, dt, t_final, time, Ie, t_pulse, start_current, end_current):
  print('Checking Simulation Setup... ')
  try:
    check_Vrest = Vrest
    check_Vinitial = Vinitial
    check_t0 = t0
    check_dt = dt
    check_t_final = t_final
    check_time = time
    check_Ie = Ie
    check_t_pulse = t_pulse
    check_start_current = start_current
    check_end_current = end_current
  except:
    return 0
  else:
    vals = [Vrest, Vinitial, t0, dt, t_final, Ie, t_pulse, start_current, end_current]
    correct_vals = [-70, -70, 0, 1, 500, 1.75, 300, 100, 400]
    if ft.reduce(lambda i, j : i and j, map(lambda m, k: m == k, vals, correct_vals),  False):  
      if len(time) != 500 or time[0] != 0 or time[-1] != 499:
        return 0
  return 1

def checkValues():
  chk = 3
  if checkVoltageParameters(Vrest, Vth, Vreset, Vspike) < 1:
    print('FAIL\n')
    chk = chk - 1
  else:
    print('PASS\n')
  if checkNeuronProperties(Rm, Cm, tau_m) < 1:
    print('FAIL\n')
    chk = chk - 1  
  else:
    print('PASS\n')
  if checkSimulationSetup(Vrest, Vinitial, EL, t0, dt, t_final, time, Ie, t_pulse, start_current, end_current) < 1:
    print('FAIL\n')
    chk = chk - 1
  else:
    print('PASS\n')
  return chk

try:
  check_sys = sys
except:
  printError()
else:
  modulename = 'functools'
  if modulename not in sys.modules:
    printError()
  else:
    printSuccess('INIT')

## Walkthrough
### Membrane Equation
Recall that the membrane equation is expressed as follows:

$ \displaystyle \tau_{m}\frac{dV}{dt} = E_{L} - V(t) + R_{m}I_{e} $




### Voltage Parameters

Start by defining the voltage parameters that determine the *resting, threshold, spike*, and *reset* membrane potential values.

*   $ V_{rest} = -70\;$*mV*
*   $ V_{th} = -54\;$*mV*
*   $ V_{reset} = -80\;$*mV*
*   $ V_{spike} = 0\;$*mV*

In [ ]:
# [BLOCK TAG: VP1]

try:
  check_BlockSet = blockSet
except:
  print('ERROR: BLOCK TAG: VP1 executed out of sequence. Missing BLOCK TAG: INIT')
else:
  try: 
    ##################################################################################
    # Voltage Parameters - Units mV (1 mV = 1e-3 Volts)
    Vrest = -70
    Vth = -54
    Vreset = -80
    Vspike = 0
    ##################################################################################
  except:
    printError()
  else:
    printSuccess('VP1')

### Neuron Properties
Next, we define some properties of the model neuron.  This includes values for the *membrane resistance* $ R_{m} $, the *membrane capacitance*, $ C_{m} $.  Then, using these values we can calculate the *membrane time constant*, $\tau_{m}$.  Last, we set the *reversal potential*, $ E_{L} $, to be equal to the neuron at rest, $V_{rest}$.

*    $ R_{m} = 10\;M\Omega $
*    $ C_{m} = 1\;$*nF*
*    $ \tau_{m} = R_{m}C_{m} = 10\;$*ms*
*    $ E_{L}  = V_{rest} = -70\;$*mV*


In [ ]:
# [BLOCK TAG: NP1]

try:
  ##################################################################################
  # Total Membrane Resistance - Units M_Ohm (1 M_Ohm = 1e6 Ohms)
  Rm = 10

  # Total Membrane Capacitance - Units nF (1 nF = 1e-9 Farads)
  Cm = 1

  # Membrane Time Constant - Units ms (1 ms = 1e-3 Seconds)
  tau_m = Rm*Cm

  # Leakage Conductance Reversal Potential
  EL = Vrest
  ##################################################################################
except:
  printError()
else:
  printSuccess('NP1')

### Simulation Setup
We want to run a simulation of the integrate-and-fire model neuron in order to observe the evolution of the membrane potential $V$ overtime.  We begin with the neuron at resting conditions: $V = V_{rest} = -70\;$*mV*.




In [ ]:
# [BLOCK TAG: SS1]

try:
  ##################################################################################
  # Initial Voltage
  Vinitial = Vrest
  ##################################################################################
except:
  printError()
else:
  printSuccess('SS1')

We will be running a 500 ms simulation.  The following lines of code setup a time span for the simulation.  This is simply a matter of defining the start time $t_{0} = 0$ and the total length (in ms) of the simulation: $t_{final} = 500$.  

Throughout the simulation, we calculate the membrane potential $V$ at each *time-step*.  The time-step is the change in time for each iteration of the simulation, for example if $t_{0} = 0$, the next computation of $V$ is performed at $t_{0} + dt$.  

Thus, by setting $dt = 1$ (in ms), the simulation will compute $V$ at time $t = 1, 2, \ldots, t_{final}$.  For $dt = 1$, we compute $V$ at every 1 ms until the end of the simulation is reached.

In [ ]:
# [BLOCK TAG: SS2]

try:
  ##################################################################################
  # Simulation Time Span (0 to 500ms, dt = 1ms)
  t0 = 0
  dt = 1
  t_final = 500

  # What does the range() function do?
  time = range(t0, t_final, dt)
  ##################################################################################
except:
  printError()
else:
  printSuccess('SS2')

We must define one last parameter, and that is the injected current $I_{e}$.  There are two aspects of this parameter that must be defined: the strength of the current measured in nanoamperes (nA), and the time values at which the current is switched on and off.

Here we make use of the **numpy** library (to learn more about how to use this library, go to  https://numpy.org/doc/stable/).

We want to begin applying the injected current $I_{e}$ at 100 ms, and switch it off at 400 ms.

*    $ I_{e} = 1.75\;$*nA*
*    $ t_{pulse} = 300\;$*ms* 
*    $ start_{current} = \displaystyle \frac{\left|t_{final}-t_{pulse}\right|}{2} = \frac{\left|500-300\right|}{2} = 100\;$*ms* 
*    $ end_{current} = start_{current} + t_{pulse} = 100 + 300 = 400\;$*ms* 

In [ ]:
# [BLOCK TAG: SS3]

try:
  ##################################################################################
  # Input Current: Ie - Units nA (1 nA = 10-9 Amperes)
  Ie = 1.75

  # Current Pulse Time Span - Units ms (1 ms = 1e-3 Seconds)
  t_pulse = 300 

  # Time at which the current is applied. 
  # We have imported the NumPy library as np so can use the absolute() function.
  start_current = np.absolute(t_final-t_pulse)/2 

  # Time at which the current is switched off
  end_current = start_current+t_pulse 
  ##################################################################################
except:
  printError()
else:
  printSuccess('SS3')

### Computing and Storing $\frac{dV}{dt}$

We are about ready to finish the code implementation for simulating an integrate-and-fire model neuron.

We need some way to store the values of the membrane potential $V$ at each time step.  To do this, we simply create an empty list $V[t]$ with a length equal to the number of time-steps of our simulation.  We then set $V[0] = V_{initial} $ as we begin our simulation at resting conditions.

In [ ]:
# [BLOCK TAG: CS1]

try:
  ##################################################################################
  # Create a list V[t] to store the value of V at each time-step dt
  V = [0] * len(time)

  # Set the initial value at time t = t0 to the initial value Vinitial
  V[0] =  Vinitial
  ##################################################################################
except:
  printError()
else:
  printSuccess('CS1')

Finally, we run our simulation according to the following *psuedocode*.

---

*for each time-step from $t = t_{0}$ to $t = t_{final}$*
> *If the current time $t < start_{current}\ $ or $\ t > end_{current}$*
>> $I_{e} = 0$

> *otherwise*
>> $I_{e} = 1.75\ nA$

> *Use Euler's Method of Numerical Integration* 

> $\displaystyle dV = \frac{dt}{\tau_{m}}(E_{L} - V[t] + R_{m}I_{e})$

> *Update $V[t+1] = V[t] + dV$*

> *If $\ V[t+1] \geq V_{th} $* 
>> $V[t] = V_{spike}$

>> $V[t+1] = V_{reset}$

*end*

---

This translates to the following Python code.

In [ ]:
# [BLOCK TAG: CS2]

try:
  chk = checkValues()
except:
  printError()
else:
  try:
    ##################################################################################
    # For each timestep we compute V and store the value
    for t in time[0:-2]:

        # If time t < 100 ms, we do not turn the Injected Current on yet.
        # If time t > 400ms, we turn the Injected Current off.
        if t < start_current or t > end_current:
            ie = 0
        # Otherwise, we switch on the Injected Current 
        else:
            ie = Ie

        # Using Euler's Method for Numerical Integration (See Chapter Text)
        # we compute the change in voltage dV as follows (using the model equation)
        dV = (dt/tau_m)*(EL - V[t] + Rm*ie)

        # Store this new value into our list
        V[t+1] = V[t] + dV

        # If the voltage value we computed crosses the threshold value Vth, a spike occurs.
        if V[t+1] >= Vth:
            # In the event of a spike, the membrane potential spikes up to Vspike mV
            V[t] = Vspike
            # Followed by immediate hyperpolarization to its Vreset value.
            V[t+1] = Vreset
    ##################################################################################     
  except:
    printError()
  else:
    if chk == 3:
      printSuccess('CS2')
    else:
      printError()

### Visualizing Results

Now we have values of $V$ for each time-step of the simulation, we can visualize the results by using Python to plot the data.  This makes use of another widely used library **plotly** (to learn more about plotting data with this library, go to https://plotly.com/python/reference/index/).

In [ ]:
# [BLOCK TAG: VR1]

try:
  if 'CS2' not in blockSet:
    print('ERROR: BLOCK TAG: VP1 executed out of sequence. Missing BLOCK TAG: CS2')
  else:
    try:
      ##################################################################################
      # Data 
      x = list(time[0:-2])
      y = V

      # Plot data
      fig = go.Figure(data=go.Scatter(x=x, y=y))

      fig.add_shape(type='line',
                          x0=min(x),
                          y0=-54,
                          x1=max(x),
                          y1=-54,
                          line=dict(color='Red'),
                          line_dash='dash')

      # Update xaxis properties
      fig.update_xaxes(title_text='Time t (ms)')

      # Update yaxis properties
      fig.update_yaxes(title_text='Membrane Potential V (mV)', range=[-90,10])

      # Update title and size
      fig.update_layout(height=450, width=700, 
                                      title_text='Integrate-and-Fire Model Neuron',
                                      showlegend = False)

      # Update theme
      fig.layout.template = 'plotly_dark'

      # Show figure
      fig.show()
      ##################################################################################

      printSuccess('VR1')
    except:
      printError()
except:
  printError()

## Integrate-and-Fire Spiking Neuron Model - Full Code

In [ ]:
import numpy as np
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Voltage Paramaters - Units mV (1 mV = 1e-3 Volts)
Vrest = -70
Vth = -54 
Vreset = -80
Vspike = 0

# Total Membrane Resistance - Units M_Ohm (1 M_Ohm = 1e6 Ohms)
Rm = 10

# Total Membrane Capacitance - Units 
Cm = 1

# Membrane Time Constant - Units ms (1 ms = 1e-3 Seconds)
tau_m = Rm*Cm

# Initial V
Vinitial = Vrest
EL = Vrest

# Simulation Time Span (0 to 500ms, dt = 1ms)
t0 = 0
dt = 1
t_final = 500

# Create the list of time step values
time = range(t0, t_final, dt)

# Current Pulse Time Span - Units ms (1 ms = 1e-3 Seconds)
t_pulse = 300 

# Time at which the current is applied
start_current = np.absolute(t_final-t_pulse)/2 

# Time at which the current is switched off
end_current = start_current+t_pulse 

# Input Current: Ie - Units nA (1 nA = 10-9 Amperes)
# Using 1.75 for default
Ie = 1.75

# Create a list V(t) to store the value of V at each time-step dt
V = [0] * len(time)

# Set the initial value at time t = t0 to the resting potential
V[0] = Vrest

# For each timestep we compute V and store the value
for t in time[0:-2]:

    # If time t < 100 ms, we do not turn the Injected Current on yet.
    # If time t > 400ms, we turn the Injected Current off.
    if t < start_current or t > end_current:
        ie = 0
    # Otherwise, we switch on the Injected Current 
    else:
        ie = Ie

    # Using Euler's Method for Numerical Integration (See Chapter Text)
    # we compute the change in voltage dV as follows (using the model equation)
    dV = (dt/tau_m)*(EL - V[t] + Rm*ie)

    # Store this new value into our list
    V[t+1] = V[t] + dV

    # If the voltage value we computed crosses the threshold value Vth, a spike occurs.
    if V[t+1] >= Vth:
        # In the event of a spike, the membrane potential spikes up to Vspike mV
        V[t] = Vspike
        # Followed by immediate hyperpolarization to its Vreset value.
        V[t+1] = Vreset

# Data 
x = list(time[0:-2])
y = V

# Plot data
fig = go.Figure(data=go.Scatter(x=x, y=y))

fig.add_shape(type='line',
                    x0=min(x),
                    y0=-54,
                    x1=max(x),
                    y1=-54,
                    line=dict(color='Red'),
                    line_dash='dash')

# Update xaxis properties
fig.update_xaxes(title_text='Time t (ms)')

# Update yaxis properties
fig.update_yaxes(title_text='Membrane Potential V (mV)', range=[-90,10])

# Update title and size
fig.update_layout(height=450, width=700, 
                                title_text='Integrate-and-Fire Model Neuron',
                                showlegend = False)

# Update theme
fig.layout.template = 'plotly_dark'

# Show figure
fig.show()

## Coding Challenge Problems

In [ ]:
#@title Run Simulation
#@markdown Execute the code block and use the sliders to set values in order to answer the Coding Challenge Problems in the chapter text.

#@markdown (Tip: Select a slider and use the left and right arrow keys to slide to the desired value.)
import numpy as np
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import ipywidgets as widgets

# Voltage Paramaters - Units mV (1 mV = 1e-3 Volts)
Vrest = -70
Vth = -54 
Vreset = -80
Vspike = 0

# Total Membrane Resistance - Units M_Ohm (1 M_Ohm = 1e6 Ohms)
Rm = 10

# Total Membrane Capacitance - Units 
Cm = 1

# Membrane Time Constant - Units ms (1 ms = 1e-3 Seconds)
tau_m = Rm*Cm

# Initial V
Vinitial = Vrest
EL = Vrest

# Simulation Time Span (0 to 500ms, dt = 1ms)
t0 = 0
dt = 1
t_final = 500

# Create the list of time step values
time = range(t0, t_final, dt)

# Current Pulse Time Span - Units ms (1 ms = 1e-3 Seconds)
t_pulse = 300 

# Time at which the current is applied
start_current = np.absolute(t_final-t_pulse)/2 

# Time at which the current is switched off
end_current = start_current+t_pulse 

# Input Current: Ie - Units nA (1 nA = 10-9 Amperes)
# Using 1.75 for default
Ie = 1.75

# Create a list V(t) to store the value of V at each time-step dt
V = [0] * len(time)

# Set the initial value at time t = t0 to the resting potential
V[0] = Vrest

def compute_iaf_neuron(Ie, Rm, Cm, tau_m):
  # For each timestep we compute V and store the value
  for t in time[0:-2]:

      # If time t < 100 ms, we do not turn the Injected Current on yet.
      # If time t > 400ms, we turn the Injected Current off.
      if t < start_current or t > end_current:
          ie = 0
      # Otherwise, we switch on the Injected Current 
      else:
          ie = Ie

      # Using Euler's Method for Numerical Integration (See Chapter Text)
      # we compute the change in voltage dV as follows (using the model equation)
      dV = (dt/tau_m)*(EL - V[t] + Rm*ie)

      # Store this new value into our list
      V[t+1] = V[t] + dV

      # If the voltage value we computed crosses the threshold value Vth, a spike occurs.
      if V[t+1] >= Vth:
          # In the event of a spike, the membrane potential spikes up to Vspike mV
          V[t] = Vspike
          # Followed by immediate hyperpolarization to its Vreset value.
          V[t+1] = Vreset
  return V

def plot_iaf_neuron(V, time):
  # Data 
  x = list(time[0:-2])
  y = V

  # Plot data
  fig = go.Figure(data=go.Scatter(x=x, y=y))

  fig.add_shape(type='line',
                      x0=min(x),
                      y0=-54,
                      x1=max(x),
                      y1=-54,
                      line=dict(color='Red'),
                      line_dash='dash')

  # Update xaxis properties
  fig.update_xaxes(title_text='Time t (ms)')

  # Update yaxis properties
  fig.update_yaxes(title_text='Membrane Potential V (mV)', range=[-90,10])

  # Update title and size
  fig.update_layout(height=450, width=700, 
                                  title_text='Integrate-and-Fire Model Neuron',
                                  showlegend = False)

  # Update theme
  fig.layout.template = 'plotly_dark'

  # Show figure
  fig.show()

style = {'description_width':'auto'}
@widgets.interact(
  Ie = widgets.FloatSlider(
    value=1.75,
    min=1.50,
    max=2.50,
    step=0.01,
   description='Ie',
    style = style,
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='1.2f'
  ),
  Rm = widgets.IntSlider(
    value=10,
    min=5,
    max=20,
    step=1,
    description='Rm',
    style = style,
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
  ),
  Cm = widgets.IntSlider(
    value=1,
    min=1,
    max=5,
    step=1,
    description='Cm',
    style = style,
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
  )
)
def simulate_iaf_neuron(Ie=1.75, Rm=10, Cm=1):
  tau_m = Rm*Cm
  V = compute_iaf_neuron(Ie, Rm, Cm, tau_m)
  plot_iaf_neuron(V, time)